In [ ]:
# pip install -U pip setuptools wheel
# pip install -U spacy
# python -m spacy download en_core_web_sm
# pip install allennlp==0.9.0

In [ ]:
# pip install allennlp==0.9.0

In [1]:
import thesis_helper
functions = thesis_helper.Thesis_Helper()
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:


import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from flair.data import Sentence

from flair.embeddings import ELMoEmbeddings

# init embedding
embedding = ELMoEmbeddings('small')

In [4]:
annotations = '/Users/ivowings/Sync/Thesis/Datasources/Preprocessed/Combined/Taxonomy/Normal/Annotated/combined_annotations.csv'

In [5]:
df = pd.read_csv(annotations,sep=';')

#Filling any empty context columns with 'empty'
df['left_context'] = df['left_context'].fillna('empty')
df['right_context'] = df['right_context'].fillna('empty')

df['concatenated'] = df['left_context'] + ' | ' + df['candidate_skill'] + ' | ' + df['right_context']
print('Number of annotated rows ',df.shape[0])

Number of annotated rows  20740


In [ ]:
#1 token has 1536 columns
# there are more than 3 tokens
def ELMo_embedder(text):

    string = Sentence(text)
    embedding.embed(string)

    # Creating a list which stores the indexes of the | symbols
    bar_indexes = []
    #Creating a list which stores the embedding_tensors
    embedding_tensors = []

    #Checking the sentence object for the | symbols and storing their indexes
    for x in range(1,len(string)+1):
        if '|' in str(string.get_token(x)):
            bar_indexes.append(x)

    #Collecting the embeddings for every index between the indexes in bar_indexes
    word_embedding_indexes = range(bar_indexes[0]+1,bar_indexes[1])
    for x in word_embedding_indexes:
        embedding_tensors.append(pd.Series(string[x].embedding))
        embedding_tensors.append(222)

    #Removing last 666 from list
    embedding_tensors.pop()

    #Turning the elements from embedding_tensors into dataframe rows
    row = pd.DataFrame()
    for x in range(0,len(embedding_tensors)):
        row = row.append(pd.DataFrame(pd.Series(embedding_tensors[x])))

    row = row.transpose().reset_index(drop=True)
    
    #Changing the column names in order to make pd.concat work later
    row.columns = [x for x in range(0,len(row.columns))]
    return row

In [ ]:
df['embeddings'] = df['concatenated'].progress_apply(ELMo_embedder)

In [ ]:
x_elmo = pd.concat(df['embeddings'].tolist()).reset_index(drop=True)
x_elmo = x_elmo.fillna(0)

In [ ]:
import pickle
pickle_out = open(('x_elmo'+".pickle"),"wb")
pickle.dump(x_elmo, pickle_out)
pickle_out.close()

import pickle
pickle_out = open(('y_elmo'+".pickle"),"wb")
pickle.dump(df['label'], pickle_out)
pickle_out.close()

In [2]:
x_elmo = functions.load_object('x_elmo')
y_elmo = functions.load_object('y_elmo')

In [ ]:
# %%time
# from sklearn.linear_model import LogisticRegression
# clf = LogisticRegression(solver='liblinear', max_iter=100000000000000000,random_state=123)#.fit(x_elmo, y_elmo)
# #clf.score(x_elmo, y_elmo)

In [ ]:
# %%time
# #2 folds took 23 minutes with lbfgs solver
# #default took 19 min
# #liblinear took 
# solvers = ['sag', 'saga', 'lbfgs', 'liblinear', 'newton-cg']

# from sklearn.model_selection import cross_validate
# from statistics import mean
# performance_measures = ['precision_macro', 'recall_macro','f1_macro']
# scores = cross_validate(clf, x_elmo, y_elmo, scoring=performance_measures,cv=2,verbose=1,n_jobs=-1)#n_jobs=0, verbose=1)

# average_precision = mean(scores['test_precision_macro'])
# average_recall = mean(scores['test_recall_macro'])
# average_f1 = mean(scores['test_f1_macro'])

In [ ]:
# scores['test_precision_macro']

In [3]:
%%time

functions.model_performance(x_elmo, y_elmo)

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 159.5min finished
 17%|█▋        | 1/6 [2:39:31<13:17:36, 9571.31s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 48.6min finished
 33%|███▎      | 2/6 [3:28:06<6:17:02, 5655.66s/it] 

...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 32.4min finished
 83%|████████▎ | 5/6 [4:04:32<32:30, 1950.73s/it]  

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 213.9min finished
100%|██████████| 6/6 [7:38:26<00:00, 4584.43s/it]


CPU times: user 1.9 s, sys: 3.02 s, total: 4.92 s
Wall time: 7h 38min 26s


,Classifier,Precision,Recall,F1
0,LR,0.761868,0.727245,0.730439
1,GBC,0.736234,0.614475,0.642147
2,SGD,NaN,NaN,NaN
3,RF,0.882080,0.579037,0.624879
4,SVM,0.534333,0.426132,0.437950
5,MLP,0.715426,0.721307,0.707554


In [6]:
df['pos'] = df['candidate_skill'].progress_apply(functions.pos_tagger)
df['pos'] = df['pos'].progress_apply(functions.sequence_counter)

pos_dicts = df[['pos']]
pos_dicts = pos_dicts['pos'].apply(pd.Series)
pos_dicts = pos_dicts.fillna(0).astype(int)

df['dep'] = df['candidate_skill'].progress_apply(functions.dep_tagger)
df['dep'] = df['dep'].progress_apply(functions.sequence_counter)

dep_dicts = df[['dep']]
dep_dicts = dep_dicts['dep'].apply(pd.Series)
dep_dicts = dep_dicts.fillna(0).astype(int)

x_pos = pos_dicts.join(dep_dicts,lsuffix='_gram', rsuffix='_pos')

x = x_elmo.join(x_pos, lsuffix='_embedding', rsuffix='_pos')
print(x.shape[1])

100%|██████████| 20740/20740 [01:52<00:00, 184.47it/s]


3134


In [7]:
%%time

functions.model_performance(x.fillna(0), y_elmo)

  0%|          | 0/6 [00:00<?, ?it/s]

Starting model evaluation
We are at classifier  LogisticRegression(max_iter=10000000000000000000000, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 148.8min finished
 17%|█▋        | 1/6 [2:28:45<12:23:49, 8925.87s/it]

We are at classifier  GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=456)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 47.9min finished
 33%|███▎      | 2/6 [3:16:38<5:57:41, 5365.31s/it] 

...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 32.5min finished
 83%|████████▎ | 5/6 [3:53:03<31:29, 1889.58s/it]  

We are at classifier  MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,),
              max_iter=10000000000000000000000, random_state=456,
              solver='lbfgs')


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 711.0min finished
100%|██████████| 6/6 [15:44:04<00:00, 9440.83s/it] 

CPU times: user 2.2 s, sys: 2.65 s, total: 4.84 s
Wall time: 15h 44min 5s


,Classifier,Precision,Recall,F1
0,LR,0.764356,0.728036,0.731294
1,GBC,0.738347,0.634522,0.664286
2,SGD,NaN,NaN,NaN
3,RF,0.878428,0.576704,0.620804
4,SVM,0.534181,0.426250,0.438054
5,MLP,0.707254,0.710939,0.699292


In [ ]:
x[x.isna().any(axis=1)].index

In [ ]:
functions.print(df.concatenated.iloc[0])

In [ ]:
functions.print(x.iloc[188])

In [ ]:
%%time
functions.model_performance(x, df['label'])

In [ ]:
functions.confusion_matrix(functions.LR,x.fillna(0), df['label'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['label']), df['label'], test_size=0.20, random_state=42)



#y_test['predictions'] = functions.LR.fit(X_train.fillna(0),y_train).predict(X_test)


In [ ]:
X_train = pd.concat(X_train['test'].tolist()).reset_index(drop=True)
X_train = X_train.fillna(0)

In [ ]:
X_test = pd.concat(X_test['test'].tolist()).reset_index(drop=True)
X_test = X_test.fillna(0)

In [ ]:
y_test['predictions'] = functions.LR.fit(X_train,y_train).predict(X_test)

In [ ]:
y_train['type'] = 'train'
y_test['type'] = 'test'

y = pd.concat([y_train,y_test])

In [ ]:
test = df.join(y_test,lsuffix='_echt', rsuffix='_pred')
test = test[['left_context','recovered_gram', 'right_context', 'label_echt', 'label_pred']]
test.to_csv('/Users/ivowings/Desktop/test1000.csv')

In [ ]:
performance = df[['left_context', 'recovered_gram', 'right_context', 'label', 'predictions']]
performance

In [ ]:
performance[(performance.label==0) & (performance.predictions==1)].to_csv('/Users/ivowings/Desktop/test5.csv')

In [ ]:
indexes = [1379411,
2837,
6853,
5332,
1379216,
          6161,6921]




In [ ]:

performance = performance.loc[indexes] 
#performance = performance.rename(columns={'focus_words':'candidate_skill'})


performance = performance.replace(0,'not skill')
performance = performance.replace(1,'soft skill')
performance = performance.replace(2,'hard skill')
performance

In [ ]:
import dataframe_image as dfi
dfi.export(performance,"Predictions.png")

In [ ]:
predictions = performance.loc[indexes]

In [ ]:
predictions = predictions.replace(0,'not skill')
predictions = predictions.replace(1,'soft skill')
predictions = predictions.replace(2,'hard skill')
predictions = predictions.rename(columns={'label': 'actual_label', 'predictions':'classifier_prediction'})
predictions